In [12]:
import GPUtil
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% |  2% |


In [2]:
from gklearn.utils.graphfiles import loadDataset
import networkx as nx
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import torch
import numpy as np
import pickle as pkl
import torch
import torch.nn as nn
import torch.nn.functional as F
import svd
import rings
from svd import iterated_power as compute_major_axis
import random
from sklearn.model_selection import train_test_split
#from sklearn import datasets
from torch.utils.data import DataLoader, TensorDataset
from triangular_losses import TriangularConstraint as triangular_constraint
import torch.autograd.profiler as profiler

In [13]:
from __future__ import print_function
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

cuda:0


In [14]:
def label_to_color(label):
    if label == 'C':
        return 0.1
    elif label == 'O':
        return 0.8
    
def nodes_to_color_sequence(G):
    return [label_to_color(c[1]['label'][0]) for c in G.nodes(data=True)]

Gs,y = loadDataset('/home/ines/Documents/M2/Stage/stage_ged/Ines/DeepGED/MAO/dataset.ds')
#for e in Gs[13].edges():
#    print(Gs[13][e[0]][e[1]]['bond_type'])
print('edge max label',max(max([[G[e[0]][e[1]]['bond_type'] for e in G.edges()] for G in Gs])))
G1 = Gs[1]
G2 = Gs[9]

'''
plt.figure(0)
nx.draw_networkx(G1,with_labels=True,node_color = nodes_to_color_sequence(G1),cmap='autumn')

plt.figure(1)
nx.draw_networkx(G2,with_labels=True,node_color = nodes_to_color_sequence(G2),cmap='autumn')

plt.show()
'''

import extended_label
for g in Gs:
    extended_label.compute_extended_labels(g)
#for v in Gs[10].nodes():
#        print(Gs[10].nodes[v])

#print(nx.to_dict_of_lists(Gs[13]))



#dict={'C':0,'N':1,'O':2}
#A,labels=from_networkx_to_tensor2(Gs[13],dict)
#print(A)
#A1=(A==torch.ones(13,13)).int()
#A2=(A==2*torch.ones(13,13)).int()
#print(A1)
#print(A2)


68
edge max label 3


In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import svd
from svd import iterated_power as compute_major_axis

#torch.autograd.set_detect_anomaly(True)

class Net(nn.Module):
    
        
    def __init__(self,GraphList,normalize=False,node_label='label'):
        super(Net, self).__init__()   
        self.normalize=normalize
        self.node_label=node_label
        dict,self.nb_edge_labels=self.build_node_dictionnary(GraphList)
        self.nb_labels=len(dict)
        print(self.nb_edge_labels)
        self.device='cuda' if torch.cuda.is_available() else 'cpu'
        nb_node_pair_label=self.nb_labels*(self.nb_labels-1)/2.0
        nb_edge_pair_label=int(self.nb_edge_labels*(self.nb_edge_labels-1)/2)
        
        
        self.node_weighs=nn.Parameter(torch.tensor(1.0/(nb_node_pair_label+nb_edge_pair_label+2))+(1e-3)*torch.rand(int(self.nb_labels*(self.nb_labels-1)/2+1),requires_grad=True,device=self.device)) # all substitution costs+ nodeIns/Del. old version: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del        
        self.edge_weighs=nn.Parameter(torch.tensor(1.0/(nb_node_pair_label+nb_edge_pair_label+2))+(1e-3)*torch.rand(nb_edge_pair_label+1,requires_grad=True,device=self.device)) #edgeIns/Del
        self.card=torch.tensor([G.order() for G in GraphList]).to(self.device)
        card_max=self.card.max()
        self.A=torch.zeros((len(GraphList),card_max*card_max),dtype=torch.int,device=self.device) 
        self.labels=torch.zeros((len(GraphList),card_max),dtype=torch.int,device=self.device) #labels des noeuds
        print("A.shape = ",self.A.shape)
        for k in range(len(GraphList)):
            A,l=self.from_networkx_to_tensor(GraphList[k],dict)             
            self.A[k,0:A.shape[1]]=A[0]
            self.labels[k,0:l.shape[0]]=l
        print('adjacency matrices',self.A)
        print('node labels',self.labels)
        print('order of the graphs',self.card)
        
    def forward(self, input):  
        self=self.to(self.device)
        input=input.to(self.device)
        ged=torch.zeros(len(input)).to(self.device) 
        node_costs,nodeInsDel,edge_costs,edgeInsDel=self.from_weighs_to_costs()
        
            
        
        #print('weighs:',self.weighs.device,'device:',self.device,'card:',self.card.device,'A:',self.A.device,'labels:',self.labels.device)
        torch.cuda.empty_cache()
        GPUtil.showUtilization(all=True)
        
        for k in range(len(input)): 
            #print(k, torch.cuda.memory_allocated())
            g1=input[k][0].to(self.device)
            g2=input[k][1].to(self.device)
            n=self.card[g1]
            m=self.card[g2]
            C=self.construct_cost_matrix(g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel)      
            #S=self.mapping_from_similarity(C,n,m)
            S=self.mapping_from_cost(C,n,m)            
            v=torch.flatten(S)
            
            v=torch.flatten(S).to(self.device)
            #v=v.to(self.device)
            S=S.detach()
            normalize_factor=1.0
            if self.normalize:
                nb_edge1=(self.A[g1][0:n*n] != torch.zeros(n*n,device=self.device)).int().sum()
                nb_edge2=(self.A[g2][0:m*m] != torch.zeros(m*m,device=self.device)).int().sum()
                normalize_factor=nodeInsDel*(n+m)+edgeInsDel*(nb_edge1+nb_edge2)
            c=torch.diag(C)
            D=C-torch.eye(C.shape[0],device=self.device)*c
            ged[k]=(.5*v.T@D@v+c.T@v)/normalize_factor
        max=torch.max(ged)
        min=torch.min(ged)
        ged=(ged-min)/(max-min)
        
        return ged
    
    def from_weighs_to_costs(self):
        
        relu=torch.nn.ReLU()
        #cn=torch.exp(self.node_weighs)
        #ce=torch.exp(self.edge_weighs)
        #cn=self.node_weighs*self.node_weighs
        #ce=self.edge_weighs*self.edge_weighs
        cn=relu(self.node_weighs)
        ce=relu(self.edge_weighs)
        #total_cost=cn.sum()+ce.sum()
        #cn=cn/total_cost #/max
        #ce=ce/total_cost
        edgeInsDel=ce[-1]
        '''
        
        #cn=torch.exp(self.node_weighs)
        #ce=torch.exp(self.edge_weighs)
        cn=self.node_weighs*self.node_weighs
        ce=self.edge_weighs*self.edge_weighs
        total_cost=cn.sum()+ce.sum()
        cn=cn/total_cost #/max
        ce=ce/total_cost
        edgeInsDel=ce[-1]
        '''

        node_costs=torch.zeros((self.nb_labels,self.nb_labels),device=self.device)
        upper_part=torch.triu_indices(node_costs.shape[0],node_costs.shape[1],offset=1,device=self.device)        
        node_costs[upper_part[0],upper_part[1]]=cn[0:-1]
        node_costs=node_costs+node_costs.T

        if self.nb_edge_labels>1:
            edge_costs=torch.zeros((self.nb_edge_labels,self.nb_edge_labels),device=self.device)
            upper_part=torch.triu_indices(edge_costs.shape[0],edge_costs.shape[1],offset=1,device=self.device)        
            edge_costs[upper_part[0],upper_part[1]]=ce[0:-1]
            edge_costs=edge_costs+edge_costs.T
            del upper_part
            torch.cuda.empty_cache()
        else:
            edge_costs=torch.zeros(0,device=self.device)
        
        return node_costs,cn[-1],edge_costs,edgeInsDel
    
    def build_node_dictionnary(self,GraphList):
        #extraction de tous les labels d'atomes
        node_labels=[]
        for G in Gs:
            for v in nx.nodes(G):
                if not G.nodes[v][self.node_label][0] in node_labels:
                    node_labels.append(G.nodes[v][self.node_label][0])
        node_labels.sort()
        #extraction d'un dictionnaire permettant de numéroter chaque label par un numéro.
        dict={}
        k=0
        for label in node_labels:
            dict[label]=k
            k=k+1
        print("node_labels : ",node_labels)
        print(dict,len(dict))
        print("nb edge labels = ",max(max([[int(G[e[0]][e[1]]['bond_type']) for e in G.edges()] for G in GraphList])))
    
        return dict,max(max([[int(G[e[0]][e[1]]['bond_type']) for e in G.edges()] for G in GraphList]))
    
    def from_networkx_to_tensor(self,G,dict):    
        A=torch.tensor(nx.to_scipy_sparse_matrix(G,dtype=int,weight='bond_type').todense(),dtype=torch.int)        
        lab=[dict[G.nodes[v][self.node_label][0]] for v in nx.nodes(G)]
   
        return (A.view(1,A.shape[0]*A.shape[1]),torch.tensor(lab))

    def construct_cost_matrix(self,g1,g2,node_costs,edge_costs,nodeInsDel,edgeInsDel):
        n = self.card[g1].item()
        m = self.card[g2].item()
        
        with torch.no_grad() : 
            A1=torch.zeros((n+1,n+1),dtype=torch.int,device=self.device)
            A1[0:n,0:n]=self.A[g1][0:n*n].view(n,n)
            A2=torch.zeros((m+1,m+1),dtype=torch.int,device=self.device)
            A2[0:m,0:m]=self.A[g2][0:m*m].view(m,m)

        
         # costs: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del
        
        #C=cost[3]*torch.cat([torch.cat([C12[l][k] for k in range(n+1)],1) for l in range(n+1)])
        #Pas bien sur mais cela semble fonctionner.
        C=edgeInsDel*self.matrix_edgeInsDel(A1,A2)
        if self.nb_edge_labels>1:
            for k in range(self.nb_edge_labels):
                for l in range(self.nb_edge_labels):
                    if k != l:
                        C.add_(self.matrix_edgeSubst(A1,A2,k+1,l+1).multiply_(edge_costs[k][l]))
                        C=C+edge_costs[k][l]*self.matrix_edgeSubst(A1,A2,k+1,l+1) #C=C+
        #C=cost[3]*torch.tensor(np.array([ [  k!=l and A1[k//(m+1),l//(m+1)]^A2[k%(m+1),l%(m+1)] for k in range((n+1)*(m+1))] for l in range((n+1)*(m+1))]),device=self.device)        
        
        l1=self.labels[g1][0:n]
        l2=self.labels[g2][0:m]
        D=torch.zeros((n+1)*(m+1),device=self.device)
        D[n*(m+1):]=nodeInsDel
        D[n*(m+1)+m]=0
        D[[i*(m+1)+m for i in range(n)]]=nodeInsDel
        D[[k for k in range(n*(m+1)) if k%(m+1) != m]]=torch.tensor([node_costs[l1[k//(m+1)],l2[k%(m+1)]] for k in range(n*(m+1)) if k%(m+1) != m],device=self.device )
        mask = torch.diag(torch.ones_like(D))
        
        #C=mask*torch.diag(D).add((1. - mask)*C)  #.multiply_(1. - mask).add_(mask.multiply_(torch.diag(D)))
        #mask.multiply_(torch.diag(D))
        #C.multiply_(1. - torch.diag(torch.ones_like(D))).add_(mask)
        C=mask*torch.diag(D) #+ (1. - mask)*C
             
        #del C
        #torch.cuda.empty_cache()

        return C
    def matrix_edgeInsDel(self,A1,A2):
        Abin1=(A1!=torch.zeros((A1.shape[0],A1.shape[1]),device=self.device))
        Abin2=(A2!=torch.zeros((A2.shape[0],A2.shape[1]),device=self.device))
        C1=torch.einsum('ij,kl->ijkl',torch.logical_not(Abin1),Abin2)
        C2=torch.einsum('ij,kl->ijkl',Abin1,torch.logical_not(Abin2))
        C12=torch.logical_or(C1,C2).int()
        #del Abin1,Abin2
        #torch.cuda.empty_cache()
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C12,1),1),0),1)

    def matrix_edgeSubst(self,A1,A2,lab1,lab2):
        Abin1=(A1==lab1*torch.ones((A1.shape[0],A1.shape[1]),device=self.device)).int()
        Abin2=(A2==lab2*torch.ones((A2.shape[0],A2.shape[1]),device=self.device)).int()
        C=torch.einsum('ij,kl->ijkl',Abin1,Abin2)
        #del Abin1,Abin2
        #torch.cuda.empty_cache()
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C,1),1),0),1).float()
    
    def similarity_from_cost(self,C):
        N=C.shape[0]
             
        #return (torch.norm(C,p='fro')*torch.eye(N,device=self.device) -C)
        return (C.max()*torch.eye(N,device=self.device) -C)
    
    def mapping_from_cost(self,C,n,m):
        c=torch.diag(C)
        D=C-torch.eye(C.shape[0],device=self.device)*c
        x0=svd.eps_assigment_from_mapping(torch.exp(-.5*c.view(n+1,m+1)),10).view((n+1)*(m+1),1)
        x=svd.franck_wolfe(x0,D,c,5,10,n,m)
        def print_grad(grad):
            if(grad.norm()!= 0.0):
                print(grad)
        
#        x.register_hook(print_grad)
        return x
    
    def mapping_from_similarity(self,C,n,m):
        M=self.similarity_from_cost(C)
        first_ev=compute_major_axis(M)
        #first_ev=self.iterated_power(M,inv=True)
        if(first_ev.sum() <0):
            first_ev=-first_ev
        S=torch.exp(first_ev.view(n+1,m+1)) # enforce the difference, accelerate the convergence. 
        S=self.eps_assigment_from_mapping(S)
        return  S
        
    def eps_assigment_from_mapping(self,S):
        ones_n = torch.ones(S.shape[0],device=S.device)
        ones_m = torch.ones(S.shape[1],device=S.device)
    
        Sk = S
        for i in range(20):
            D=torch.diag(1.0/(Sk@ones_m))
            D[D.shape[0]-1,D.shape[1]-1]=1.0
            Sk1 = D@Sk
            D=torch.diag(1.0/(ones_n@Sk1))
            D[D.shape[0]-1,D.shape[1]-1]=1.0
            Sk = Sk1@D
        
        return Sk

print('Gs=',len(Gs))
model = Net(Gs,normalize=True,node_label='extended_label')

params = list(model.parameters())
print(len(params))
print(params[0])
#print(model(input))
print(max([G.order() for G in Gs]),len(Gs))
print('toto')

Gs= 68
['C_1C', 'C_1C1C1N', 'C_1C1C2C', 'C_1C1N', 'C_1C1N2C', 'C_1C1O', 'C_1C1O2C', 'C_1C2C', 'C_1C3C', 'C_1N', 'C_1O', 'C_2C', 'C_2C2C', 'C_3C', 'N_1C', 'N_1C1C', 'N_1C1C1C', 'O_1C', 'O_1C1C']
{'C_1C': 0, 'C_1C1C1N': 1, 'C_1C1C2C': 2, 'C_1C1N': 3, 'C_1C1N2C': 4, 'C_1C1O': 5, 'C_1C1O2C': 6, 'C_1C2C': 7, 'C_1C3C': 8, 'C_1N': 9, 'C_1O': 10, 'C_2C': 11, 'C_2C2C': 12, 'C_3C': 13, 'N_1C': 14, 'N_1C1C': 15, 'N_1C1C1C': 16, 'O_1C': 17, 'O_1C1C': 18} 19
3
torch.Size([68, 729])
adjacency matrices tensor([[          0,           1,           0,  ..., -1960201184,
               32729,   565413936],
        [          0,           1,           0,  ...,       32729,
         -1889688488,       32729],
        [          0,           1,           0,  ...,   975793452,
           538976266,   538976288],
        ...,
        [          0,           1,           0,  ...,           0,
                   0,           0],
        [          0,           1,           0,  ...,           0,
               

In [ ]:
print('Gs=',len(Gs))
model = Net(Gs,normalize=True,node_label='extended_label')

nb_class1=12
nb_class2=int((nb_class1-1)/2)
train_size=nb_class1+nb_class2
#train_size=20

#if train_size % 2 == 0:
#    nb_class1=int(train_size/2)
#    nb_class2=int(train_size/2)
#else:
#    nb_class1=int(train_size/2)+1
#    nb_class2=int(train_size/2)
    
print((torch.abs(10000*torch.randn(nb_class1)).int()%class1.size()[0]).long())
random_class1=class1[(torch.abs(10000*torch.randn(nb_class1)).int()%class1.size()[0]).long()]
random_class2=class2[(torch.abs(10000*torch.randn(nb_class2)).int()%class2.size()[0]).long()]
train_graphs=torch.cat((random_class1,random_class2),0)
print('train graphs:',train_graphs)

couples=torch.triu_indices(train_size,train_size,offset=1)
print('couples=',couples)
print('nb_class1/nb_class2=',nb_class1,nb_class2)
#combinations=itertools.combinations(range(nb),2)

nb_elt=int(nb_class1*(nb_class1+2*nb_class2-1)/2)
print('couples restreints:',couples[:,0:nb_elt])

#nb_elt=int(train_size*(train_size-1)/2)
data=torch.empty((nb_elt,2),dtype=torch.int)
yt=torch.ones(nb_elt)
print('old_size, new size=',nb_elt,.5*nb_class1*(nb_class1+2*nb_class2-1))
data[0:nb_elt,0]=train_graphs[couples[0,0:nb_elt]]
data[0:nb_elt,1]=train_graphs[couples[1,0:nb_elt]]

#data[0:nb_elt,0]=train_graphs[couples[0]]
#data[0:nb_elt,1]=train_graphs[couples[1]]
print(nb_elt)
#couples=[]
for k in range(nb_elt):
    if (y[data[k][0]]!=y[data[k][1]]):
        yt[k]=-1.0        

print('data=',data)

    trainloader=torch.utils.data.DataLoader(DatasetTrain,batch_size=len(couples_train),shuffle=True,drop_last=True, num_workers=0) #128, len(couples_train)

    validationloader=torch.utils.data.DataLoader(DatasetValid, batch_size=len(couples_test_train), drop_last=True,num_workers=0) #64,128,len(couples_test_train)
    
    print(len(trainloader),len(validationloader))
    print(len(trainloader),len(validationloader))
    
    #train_D,valid_D=getting_data(train_D, valid_D,train_L,valid_L)
    torch.save(train_D, 'train_D', pickle_module=pkl) 
    torch.save(valid_D, 'valid_D', pickle_module=pkl) 
    torch.save(train_L, 'train_L', pickle_module=pkl) 
    torch.save(valid_L, 'valid_L', pickle_module=pkl) 
    
    print("valid_D = ",valid_D,len(valid_D))
    
    return trainloader,validationloader,couples_train,yt,couples_test_train,yv

In [52]:
print(device)

cuda:0
0 45.26795196533203
ged= tensor([ 0.1649,  0.0000,  0.0852,  0.0851,  0.0000,  0.2567,  0.0764,  0.1162,
         0.1841,  0.0764,  0.1979, -0.1033, -0.1074, -0.8296, -0.3208, -0.5160,
         0.1678,  0.2679,  0.2679,  0.1678,  0.3421,  0.2292,  0.3086,  0.2629,
         0.2292,  0.2893, -0.2479, -0.3502, -0.9623, -0.4204, -0.7563,  0.2199,
         0.2200,  0.0106,  0.3259,  0.1455,  0.2445,  0.2368,  0.1455,  0.2167,
        -0.1368, -0.0780, -0.9908, -0.3927, -0.7526,  0.1583,  0.1759,  0.2920,
         0.1801,  0.2041,  0.2319,  0.1801,  0.2914, -0.2370, -0.3864, -0.8093,
        -0.3636, -0.5313,  0.1760,  0.2919,  0.1801,  0.2041,  0.2319,  0.1801,
         0.2914, -0.2371, -0.3864, -0.8093, -0.3636, -0.5313,  0.3259,  0.1455,
         0.2445,  0.2368,  0.1455,  0.2167, -0.1368, -0.0780, -0.9908, -0.3927,
        -0.7526,  0.1661,  0.3042,  0.2129,  0.1661,  0.3351, -0.2895, -0.5914,
        -0.7142, -0.3205, -0.5445,  0.2482,  0.1499,  0.0180,  0.2761, -0.1933,
        

In [14]:
#if torch.cuda.device_count() > 1:
#  print("Let's use", torch.cuda.device_count(), "GPUs!")
  # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
#  model = nn.DataParallel(model)

    criterion = torch.nn.HingeEmbeddingLoss(margin=1.0,reduction='sum')
    criterionTri=triangular_constraint()
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
    print(device)

    #whole_input=data.to(device)
    
    train_input=couples_train.to(device)
    #valid_input=couples_test_train.to(device)
    
    target=yt.to(device) 
    InsDel=np.empty((nb_iter,2))
    node_costs,nodeInsDel,edge_costs,edgeInsDel=model.from_weighs_to_costs()
    nodeSub=np.empty((nb_iter,int(node_costs.shape[0]*(node_costs.shape[0]-1)/2)))
    edgeSub=np.empty((nb_iter,int(edge_costs.shape[0]*(edge_costs.shape[0]-1)/2)))
    loss_plt=np.empty(nb_iter)
    for t in range(nb_iter):    
        # Forward pass: Compute predicted y by passing data to the model    


        y_pred = model(input).to(device)
        
        
        # Compute and print loss
        loss = criterion(y_pred, target).to(device)    
        node_costs,nodeInsDel,edge_costs,edgeInsDel=model.from_weighs_to_costs()
        triangularInq=criterionTri(node_costs,nodeInsDel,edge_costs,edgeInsDel)
        loss=loss*(1+triangularInq)
        loss.to(device)
        InsDel[t][0]=nodeInsDel.item()
        InsDel[t][1]=edgeInsDel.item()
        
        k=0
        for p in range(node_costs.shape[0]):
            for q in range(p+1,node_costs.shape[0]):
                nodeSub[t][k]=node_costs[p][q]
                k=k+1
        k=0
        for p in range(edge_costs.shape[0]):
            for q in range(p+1,edge_costs.shape[0]):
                edgeSub[t][k]=edge_costs[p][q]
                k=k+1
        if t % 100 == 99 or t==0:   
            print('ged=',y_pred*target)
            print('Distances: ',y_pred)
            print('Loss Triangular:',triangularInq.item())
            print('node_costs :')
            print(node_costs)
            print('nodeInsDel:',nodeInsDel.item())
            print('edge_costs :')
            print(edge_costs)
            print('edgeInsDel:',edgeInsDel.item())
        
            
        print(f'Iteration {t+1} \t\t Validation Loss: {valid_loss/len(validationloader)}')
        if min_valid_loss > valid_loss:
            print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f})')
            min_valid_loss = valid_loss
            iter_min_valid_loss = t
            nodeSub_min = node_costs
            edgeSub_min = edge_costs
            nodeInsDel_min = nodeInsDel
            edgeInsDel_min = edgeInsDel
        
        del valid_loss,loss
        torch.cuda.empty_cache()                
            
    print('iter and min_valid_loss = ',iter_min_valid_loss, min_valid_loss)
    print(' Min cost for nodeInsDel = ', nodeInsDel_min)
    print(' Min cost for edgeInsDel = ', edgeInsDel_min)
    print(' Min cost for nodeSub = ', nodeSub_min)
    print(' Min cost for edgeSub = ', edgeSub_min)
    torch.save(nodeInsDel_min, 'nodeInsDel_min', pickle_module=pkl) 
    torch.save(edgeInsDel_min, 'edgeInsDel_min', pickle_module=pkl) 
    torch.save(nodeSub_min, 'nodeSub_min', pickle_module=pkl) 
    torch.save(edgeSub_min, 'edgeSub_min', pickle_module=pkl)
    return InsDel,nodeSub,edgeSub,loss_plt,loss_valid_plt,loss_train_plt



In [15]:
print(len(Gs))
nb_iter=100

nb_iter=100
InsDel, nodeSub,edgeSub,loss_plt=classification(model,data,yt,nb_iter)
plt.figure(0)
plt.plot(InsDel[0:nb_iter,0],label="node")
plt.plot(InsDel[0:nb_iter,1],label="edge")
plt.title('Node/Edge insertion/deletion costs')
plt.legend()
plt.figure(1)
for k in range(nodeSub.shape[1]):
    plt.plot(nodeSub[0:nb_iter,k])
plt.title('Node Substitutions costs')
plt.figure(2)
for k in range(edgeSub.shape[1]):
    plt.plot(edgeSub[0:nb_iter,k])
plt.title('Edge Substitutions costs')
plt.figure(3)
plt.plot(loss_plt)
plt.title('Evolution of the train loss (loss_plt)')

plt.figure(4)
plt.plot(loss_valid_plt)
plt.title('Evolution of the valid loss')
'''
plt.figure(5)
plt.plot(loss_train_plt)
plt.title('Evolution of the loss_train_plt')
'''
plt.show()
plt.close()

68
1 1
1 1
valid_D =  [12, 26, 14, 9, 5, 31, 45] 7
0 308224
| ID | Name             | Serial | UUID                                     || GPU temp. | GPU util. | Memory util. || Memory total | Memory used | Memory free || Display mode | Display active |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  0 | GeForce RTX 2080 | [N/A]  | GPU-e9bd11db-4bb9-a629-cf91-fb9c04a39d9f ||       33C |        1% |          15% ||       7973MB |      1187MB |      6786MB || Enabled      | Enabled        |
loss.item of the train =  0 0.5355881452560425
ged= tensor([-0.5246, -0.4053, -0.4179,  ...,  0.5896,  0.7503,  0.4284],
       device='cuda:0', dtype=torch.float64, grad_fn=<MulBackward0>)
Distances:  tensor([0.5246, 0.4053, 0.4179,  ..., 0.5896, 0.7503, 0.4284], device='cuda:0',
       grad_fn=<DivBackward0>)
Loss Triangular: 0.0
node_costs :
tensor([

In [53]:
  
torch.save(loss_plt, 'loss_plt', pickle_module=pkl) 
torch.save(loss_valid_plt, 'loss_valid_plt', pickle_module=pkl) 
torch.save(loss_train_plt, 'loss_train_plt', pickle_module=pkl) 


torch.save(InsDel,'InsDel', pickle_module=pkl) 
torch.save(edgeSub,'edgeSub', pickle_module=pkl) 
torch.save(nodeSub,'nodeSub', pickle_module=pkl) 


In [ ]:
A=torch.tensor(nx.to_scipy_sparse_matrix(Gs[0],dtype=int,weight='bond_type').todense(),dtype=torch.int) 
print(A)

In [ ]:
plt.figure(0)
plt.plot(InsDel[0:500,0],label="node")
plt.plot(InsDel[0:500,1],label="edge")
plt.title('Node/Edge insertion/deletion costs')
plt.legend()
plt.figure(1)
for k in range(nodeSub.shape[1]):
    plt.plot(nodeSub[0:500,k])
plt.title('node Substitution costs')
plt.figure(2)
for k in range(edgeSub.shape[1]):
    plt.plot(edgeSub[0:500,k])
plt.title('edge Substitution costs')
plt.show()
plt.close()